### Pipeline

#### create a polygon using arcpy geometry primitives

In [13]:
import arcpy
import psycopg2
from psycopg2 import sql
import requests

In [19]:
# create points 
points = [
    arcpy.Point(-93.23454288744209, 44.9730581479119),
    arcpy.Point(-93.23454288744209, 44.97105520487412),
    arcpy.Point(-93.23081572654691, 44.97105520487412),
    arcpy.Point(-93.23081572654691, 44.9730581479119),
]

# create polygon
polygon = arcpy.Polygon(arcpy.Array(points), sr)

# assign SR
sr = arcpy.SpatialReference(4326)

polygon_geometry = polygon.getPart(0)
polygon_geometry.spatialReference = sr

#### convert it to WKT

In [20]:
polygon_wkt = polygon.WKT
polygon_wkt

'MULTIPOLYGON (((-93.234542886999975 44.973058148000064, -93.234542886999975 44.971055205000027, -93.230815726999936 44.971055205000027, -93.230815726999936 44.973058148000064, -93.234542886999975 44.973058148000064)))'

### use psycopg2 to import the polygon to PostGIS database

In [6]:
# connect to PostGIS database
connection = psycopg2.connect(
    database="*",
    user="*",
    password="*",
    host="*"
)

In [ ]:
# create a cursor object
cursor = connection.cursor()

# Create the table with geometry column accepting MultiPolygon
table = sql.SQL("CREATE TABLE IF NOT EXISTS polygon_lab1 (id SERIAL PRIMARY KEY, geometry GEOMETRY)")
cursor.execute(table)

# Alter the table to accept MultiPolygon geometries
alter_table = sql.SQL(
    "ALTER TABLE polygon_lab1 ALTER COLUMN geometry TYPE geometry(MultiPolygon, 4326) USING geometry::geometry(MultiPolygon, 4326)"
)
cursor.execute(alter_table)

# Set up WKT format and insert the geometry with SRID assignment
insert_query = sql.SQL("INSERT INTO polygon_lab1 (geometry) VALUES (ST_SetSRID(ST_GeomFromText('{}'), 4326))").format(sql.Literal(polygon_wkt))
cursor.execute(insert_query)

# End the cursor instance properly by invoking the close method
cursor.close()


In [37]:
# check if the polygon has successfuly created

cursor.execute("SELECT COUNT(*) FROM polygon_lab1")
count = cursor.fetchone()[0]
print("Number of polygons in 'polygon_lab1':", count)


Number of polygons in 'polygon_lab1': 1


#### import the polygon as a content layer into ArcOnline

In [6]:
# Save the dictionary as a JSON file
with open(r'D:\spring2024\GIS5572\Lab1\geojson_polygon.geojson', 'w') as json_file:
    json.dump(json_obj_2, json_file)

In [7]:
import arcpy
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Connect to your ArcGIS Online organization
gis = GIS(None,'TzuyuMa', '*', verify_cert=False)

# Upload the GeoJSON file as a new item
item = gis.content.add({'type': 'GeoJson'}, 'D:\spring2024\GIS5572\Lab1\geojson_polygon.geojson')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)

Setting `verify_cert` to False is a security risk, use at your own risk.


Exception: Unable to publish item.
User 'TzuyuMa' does not have publishing privileges.
(Error Code: 400)

#### request the geojson

In [7]:
url = r'http://34.85.193.144:5000/getgeojson'
response = requests.get(url)   
response

<Response [200]>

In [8]:
#unpack request as json
with open (r'D:\spring2024\GIS5572\Lab1\geojson_polygon.geojson', 'w') as file:
    file.write(str(response.json()))
    print('ok')

ok
